In [18]:

import cv2
import numpy as np

# Define the HSV range for red color
lower_red = np.array([0, 50, 50])
upper_red = np.array([10, 255, 255])
lower_red2 = np.array([170, 50, 50])
upper_red2 = np.array([180, 255, 255])

# Load all the images and store them in a dictionary
IMAGES = {}
for i in range(8, 18):
    for j in range(0, 60, 30):
        hour = f"{i:02}_{j:02}"
        if i < 12:
            filename = f"{hour}am.png"
        elif i == 12:
            filename = f"{hour}pm.png"
        else:
            filename = f"{i-12:02}_{j:02}pm.png"
        image = cv2.imread(filename)
        IMAGES[hour] = image

        
def detect_red(img,usr_inpt):            
    # Load the image
    #img = cv2.imread('04_30pm.png')

    # Convert the image to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Threshold the image to get the red lines
    mask1 = cv2.inRange(hsv, lower_red, upper_red)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)

    # Apply some morphological operations to clean up the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours in the mask
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours on the original image
    cv2.drawContours(img, contours, -1, (0, 0, 255), 2)


    # Display the image and the number of red lines detected
    cv2.imshow('image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    print(f"Around {usr_inpt}, there are possibilities of road blockage at {len(contours)} points")

    return len(contours)

def suggest_routes(userInput, noOfBlocks):
    print(f"\n==================Generating suggestions regarding predicted traffic condition=========================\n")
    print(f"It is suggested to try different timing, you can check predictions for next 30 minutes or previous 30 minutes to plan your best route!")

# Ask the user for input
user_input = input("Enter hour between 8am and 5pm (e.g. 8am, 8_30am, 9am, etc.): ")

# Check if the input is valid
if user_input not in IMAGES:
    print("Invalid input")
else:
    # Get the corresponding image and perform blob detection
    image = IMAGES[user_input]    #this image has the exact name as user input
    length=detect_red(image,user_input)
    suggest_routes(user_input,length)
   

Enter hour between 8am and 5pm (e.g. 8am, 8_30am, 9am, etc.): 15_00
Around 15_00, there are possibilities of road blockage at 7 points

==================Generating suggestions regarding predicted traffic condition=========================

It is suggested to try different timing, you can check predictions for next 30 minutes or previous 30 minutes to plan your best route!
